In [35]:
from transformers import RobertaForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import tweepy as tw
import pandas as pd

In [36]:
consumer_key= 'dDhUnEB64ZYErOZzZ8fikHU9K'
consumer_secret= 'MFTTcO4DyqsvaXwS2i7SXLu3UMRHUeLNKYZ8MhF4yO9IQ2Io41'
access_token= '1231580456572854273-OlByhW9omNeGMXcnOWOwbDLDnVkh0W'
access_token_secret='v7Mqon3TUNbQXo1Cja76PYs5Yk3LvByJc1Lz43FQRYS62'

In [37]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

In [38]:
def dataframe(text, size = 100, lang = "en"):
    tweets = tw.Cursor(api.search ,q= text, lang=lang , tweet_mode="extended").items(size)
    tweet =[]
    for i in tweets :
        tweet.append(i.full_text)
    df = pd.DataFrame({'tweet': tweet})
    #Removing retweets:
    df = df[~df.tweet.str.contains("RT")] 
    df= df.reset_index(drop=True)
    return df

In [39]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [40]:
# FinBERT used to filter the ESG tweets

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')
esg_filter = pipeline("text-classification", model=finbert, tokenizer=tokenizer)

In [41]:
# RoBERTa used for sentiment analysis

ROBERTA = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer_sent = AutoTokenizer.from_pretrained(ROBERTA)
config_sent = AutoConfig.from_pretrained(ROBERTA)
# PT
model_sent = RobertaForSequenceClassification.from_pretrained(ROBERTA)

sentiment = pipeline("sentiment-analysis", model=model_sent, tokenizer=tokenizer_sent)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
def simple_model(text, preprocess, esg_filter, sentiment):
    text = preprocess(text)
    if esg_filter(text)[0]['label'] == 'None':
        return 0
    else:
        if sentiment(text)[0]['label'] == 'Neutral':
            return 0
        if sentiment(text)[0]['label'] == 'Positive':
            return sentiment(text)[0]['score']
        if sentiment(text)[0]['label'] == 'Negative':
            return (-1)* sentiment(text)[0]['score']

In [32]:
#Some sample testing

text0 = "The carbon consumption at Apple has been increasing at a fast pace."
print(simple_model(text0, preprocess, esg_filter, sentiment))

text1 = "I summarized McKinsey's 10 sources of competitive advantage in one image and included some examples that I think fit the description (list is not exhaustive, of course)"
print(simple_model(text1, preprocess, esg_filter, sentiment))

text2 = "Most companies suck at solving problems. So they pay McKinsey $500K+ to do it for them"
print(simple_model(text2, preprocess, esg_filter, sentiment))

text3 = "We’re here to provide tips, tricks and helpful information when you need it most. We’re available every day to answer your questions, from 5am-8pm Pacific."
print(simple_model(text3, preprocess, esg_filter, sentiment))

text4 = "I think it’s so cool that Apple is trying to reduce carbon emission by leaving certain things that most people don’t use out of the box but I can already hear the customers yelling at me about it"
print(simple_model(text4, preprocess, esg_filter, sentiment))

text5 = "HAHAHA honestly I’ve been an Apple user for years, since the 3GS actually!  but after Samsung released this ad ima go out and get me one.Reduces carbon emission my butt Apple"
print(simple_model(text5, preprocess, esg_filter, sentiment))

text6 = "I worry that apple is increasing its average wage even though this will reduce inequalities"
print(simple_model(text6, preprocess, esg_filter, sentiment))

-0.5015495419502258
0
-0.9027664661407471
0.8792818188667297
0.814257025718689
0.896986722946167
-0.7128043174743652


In [43]:
def ESG(df, preprocess, esg_filter, sentiment):
    ESG_score = 0
    k = 0
    for i in range(df['tweet'].shape[0]):
        a = simple_model(df['tweet'][i], preprocess, esg_filter, sentiment)
        if a != 0:
            k+=1
        ESG_score += a

    ESG_score = ESG_score/k
    return ESG_score

In [44]:
#Tesla:

df = dataframe("#tesla")
ESG(df, preprocess, esg_filter, sentiment)

0.3147019426027934

In [45]:
#Exxon:

df = dataframe("#exxon")
ESG(df, preprocess, esg_filter, sentiment)

-0.6853737950325012

In [49]:
#Microsoft:

df = dataframe("#Microsoft")
ESG(df, preprocess, esg_filter, sentiment)

0.5603437469555781

In [50]:
#Accenture:

df = dataframe("#Accenture")
ESG(df, preprocess, esg_filter, sentiment)

0.6623695632990669

In [54]:
#Fox:

df = dataframe("#Fox")
ESG(df, preprocess, esg_filter, sentiment)

-0.33278047541777295

In [56]:
#Tencent:

df = dataframe("#Tencent", 1000)
ESG(df, preprocess, esg_filter, sentiment)

#with size = 100 it gave 0.88
#This makes a bit more sense ~

0.2602367627620697